In [11]:
import sys
import subprocess
from pathlib import Path
import numpy as np

# try to import tifffile, install if missing
try:
    import tifffile as tiff
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tifffile"])
    import tifffile as tiff

In [12]:
import os
from pathlib import Path
notebook_dir = Path.cwd()
proj_root = notebook_dir.parent
BASE_DIR = str(proj_root / "DATA AUGMENTATION")
# change these to your folders
input_folder = BASE_DIR
output_folder = r"BASE_DIR\DA"

def save_tiff(filepath, data):
    # make sure the folder exists before saving
    Path(filepath).parent.mkdir(exist_ok=True, parents=True)
    tiff.imwrite(filepath, data, compression=None)

In [13]:
def augment_image(img, name, outdir):
    # create 5 different versions of the image
    save_tiff(outdir / (name + "_hflip.tif"), np.fliplr(img))  # horizontal flip
    save_tiff(outdir / (name + "_vflip.tif"), np.flipud(img))  # vertical flip
    save_tiff(outdir / (name + "_90.tif"), np.rot90(img))      # rotate 90 degrees
    save_tiff(outdir / (name + "_180.tif"), np.rot90(img, 2)) # rotate 180 degrees
    save_tiff(outdir / (name + "_270.tif"), np.rot90(img, 3)) # rotate 270 degrees

In [14]:
def process_tiff(filepath, input_dir, output_dir):
    # try to read the image file
    try:
        img = tiff.imread(str(filepath))
    except:
        print("Error reading " + filepath.name)
        return
    
    # get filename without extension and figure out output folder
    name = filepath.stem
    outdir = output_dir / filepath.parent.relative_to(input_dir)
    
    # check if this is a z-stack (multiple images in one file)
    if len(img.shape) == 3 and img.shape[0] > 1:
        # process each slice separately
        for i in range(img.shape[0]):
            slice_name = name + "_z" + str(i)
            augment_image(img[i], slice_name, outdir)
    else:
        # single image, process normally
        augment_image(img, name, outdir)
    
    print("Processed " + filepath.name)

In [15]:
# convert folder paths to Path objects
input_path = Path(input_folder)
output_path = Path(output_folder)

# find all tiff files in input folder
files = []
for ext in ["*.tif", "*.tiff"]:
    files.extend(input_path.rglob(ext))

# make sure we found some files
if not files:
    print("No tiff files found in " + str(input_path))
    sys.exit()

print("Processing " + str(len(files)) + " files...")

# process each file
for f in files:
    process_tiff(f, input_path, output_path)

print("Done! Output saved to " + str(output_path))

Processing 6 files...
Processed CRYO-SEM pp x30000a.tif
Processed CRYO-SEM pp x30000a_180.tif
Processed CRYO-SEM pp x30000a_270.tif
Processed CRYO-SEM pp x30000a_90.tif
Processed CRYO-SEM pp x30000a_hflip.tif
Processed CRYO-SEM pp x30000a_vflip.tif
Done! Output saved to BASE_DIR\DA
